# Classification trees

Classification trees are very similar to regression trees, except that the target variable is categorical. In regression trees, we used the mean of the target variable in each region as the prediction. In classification trees, we use the most common class in each region as the prediction. Besides the most common class, we are also interested in the proportion of each class in each region. This is because the proportion of each class in each region is a measure of the **purity** of the region. A region is pure if it contains only one class. A region is impure if it contains multiple classes.

## Gini index

Although classification accuracy is a good measure for classification performance, a popular cost function (splitting criterion) for classification trees is the [**Gini index**](https://en.wikipedia.org/wiki/Gini_coefficient), a measure of the impurity of a region (not just feature regions in machine learning, but also regions in society in economics, e.g. to measure income/wealth inequality). This is because classification accuracy is not sufficiently sensitive for building classification trees. 

The Gini index is defined as:

$$
G = 1 - \sum_{c=1}^C p_c^2 = \sum_{c=1}^C p_c (1 - p_c),
$$

where $p_c$ is the proportion of class $c$ in the region. The Gini index is zero if the region is pure, and is one if the region is impure. The Gini index is a measure of the probability of misclassification. The Gini index is also known as the **Gini impurity**.

## Entropy

Another popular cost function (splitting criterion) for classification trees is the [**entropy**](https://en.wikipedia.org/wiki/Entropy_(information_theory)), which is defined as:

$$
H = - \sum_{c=1}^C p_c \log_2 p_c,
$$

where $p_c$ is the proportion of class $c$ in the region. The entropy is zero if the region is pure, and is one if the region is impure. 

When building classification trees, either the Gini index or the entropy are typically used to evaluate the quality of a particular split, and the split that produces the lowest cost is chosen. The Gini index and the entropy are very similar, and the Gini index is slightly faster to compute.

## Classification trees for heart disease diagnosis

Get ready by importing the APIs needed from respective libraries.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt

%matplotlib inline

Set a random seed for reproducibility.

In [ ]:
np.random.seed(2022)

We use the [Heart dataset](https://github.com/pykale/transparentML/blob/main/data/Heart.csv) (click to explore) to predict whether a patient has heart disease or not. The target variable is `AHD`, which is a binary variable that indicates whether a patient has heart disease or not. 

Load the dataset and _remove the rows with missing values_. Inspect the first few rows of the dataset.

In [ ]:
heart_url = "https://github.com/pykale/transparentML/raw/main/data/Heart.csv"

heart_df = pd.read_csv(heart_url).dropna()
heart_df.head()

Take a look at the structure of the dataset.

In [ ]:
heart_df.info()

Use the [`factorize` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html) from the `pandas` library to convert categorical variables to numerical variables. This is because the `sklearn` library only accepts numerical variables. `ChestPain` is a categorical variable that indicates the type of chest pain. `Thal` is a categorical variable that indicates the type of thalassemia. 

In [ ]:
heart_df.ChestPain = pd.factorize(heart_df.ChestPain)[0]
heart_df.Thal = pd.factorize(heart_df.Thal)[0]

Get the feature matrix `X` and the target vector `y`.

In [ ]:
X = heart_df.drop("AHD", axis=1)
y = pd.factorize(heart_df.AHD)[0]

Fit a classification tree to the data using the `DecisionTreeClassifier` class from the `sklearn.tree` module. Set the `max_leaf_nodes` parameter to 6. This will limit the number of leaf nodes to 6. Set the `max_features` parameter to 3. This will limit the number of features to consider when looking for the best split to 3. It will randomly select these 3 features at each split.

In [ ]:
clf = DecisionTreeClassifier(max_leaf_nodes=6, max_features=3)
clf.fit(X, y)

Compute the accuracy of the classification tree on the training data.

In [ ]:
clf.score(X, y)

Visualise the classification tree using the `plot_tree` function from the `sklearn.tree` module. Set the `filled` parameter to `True` to colour the nodes in the tree according to the majority class in each region. Set the `feature_names` parameter to the list of feature names. Set the `class_names` parameter to the list of class names. 

In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(
    clf, filled=True, feature_names=X.columns, class_names=["No", "Yes"], fontsize=14
)
plt.show()

## Classification trees for car seat sales

Next, let us study the [Carseats dataset](https://github.com/pykale/transparentML/blob/main/data/Carseats.csv) (click to explore). In this dataset, we want to predict whether a car seat will be `High` or `Low` based on the `Sales` and `Price` of the car seat.

Load the dataset. `Sales` is a continuous variable so we recode it as a binary variable `High` by thresholding it at 8 using the `map()` function from the `pandas` library. `High` takes on a value of 'Y' if the Sales variable exceeds 8, 'N' otherwise. We also convert categorical variables to numerical variables using the `factorize` method from the `pandas` library as above. 

Inspect the first few rows of the dataset.

In [ ]:
carseats_url = "https://github.com/pykale/transparentML/raw/main/data/Carseats.csv"

carseats_df = pd.read_csv(carseats_url)
carseats_df["High"] = carseats_df.Sales.map(lambda x: "Y" if x > 8 else "N")
carseats_df.ShelveLoc = pd.factorize(carseats_df.ShelveLoc)[0]
carseats_df.Urban = carseats_df.Urban.map(lambda x: 1 if x == "Yes" else 0)
carseats_df.US = carseats_df.US.map(lambda x: 1 if x == "Yes" else 0)
carseats_df.head()

Take a look at the structure of the dataset.

In [ ]:
carseats_df.info()

Split the dataset into training (200 samples) and test sets.

In [ ]:
X2 = carseats_df.drop(["Sales", "High"], axis=1)
y2 = carseats_df.High
train_size = 200
X_train, X_test, y_train, y_test = train_test_split(
    X2, y2, train_size=train_size, test_size=X.shape[0] - train_size
)

For this dataset, let us use 'entropy' as the split criterion at each node to fit a classification tree to the training data.

In [ ]:
criteria = "entropy"
max_depth = 6
min_sample_leaf = 4
clf_gini = DecisionTreeClassifier(
    criterion=criteria, max_depth=max_depth, min_samples_leaf=min_sample_leaf
)
clf_gini.fit(X_train, y_train)
print(clf_gini.score(X_train, y_train))

Visualise the classification tree.

In [ ]:
# one attractive feature of a tree is visulization.
plt.figure(figsize=(40, 20))  # customize according to the size of your tree
plot_tree(
    clf_gini,
    filled=True,
    feature_names=X_train.columns,
    class_names=["No", "Yes"],
    fontsize=14,
)
plt.show()

Build the confusion matrix to evaluate the model in accuracy for both training and test datasets.

In [ ]:
y_pred_train = clf_gini.predict(X_train)
cm = pd.DataFrame(
    confusion_matrix(y_train, y_pred_train).T,
    index=["No", "Yes"],
    columns=["No", "Yes"],
)
print(cm)
print("Train Accuracy is ", accuracy_score(y_train, y_pred_train) * 100)


y_pred = clf_gini.predict(X_test)
cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred).T, index=["No", "Yes"], columns=["No", "Yes"]
)
print(cm)
print("Test Accuracy is ", accuracy_score(y_test, y_pred) * 100)

The test accuracy of the fitted model is significantly lower than the training accuracy, an indication of overfitting. You can go back and change the hyperparameters in the training process to reduce the dimension (flexibility) of the parameter space and reduce overfitting.

Compute more metrics such as precision, recall, f1-score, etc.

In [ ]:
print(classification_report(y_test, y_pred))

## Exercises

**1.** All the following exercises use the [Caravan](https://github.com/pykale/transparentML/blob/main/data/Caravan.csv) dataset.

Load the **Caravan** dataset and inspect the first five rows. 

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
import pandas as pd

caravan_url = "https://github.com/pykale/transparentML/raw/main/data/Caravan.csv"
caravan_df = pd.read_csv(caravan_url)
caravan_df.head()

**2.** Now, split the Caravan dataset into train and test sets using the proportion 80:20 and fit a **DecisionTreeClassifier** with $8$ **maximum leaf nodes** and $6$ **maximum features**. Calculate the testing accuracy and AUC of the fitted model. (Use $2022$ as the random seed value)

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, roc_auc_score

np.random.seed(2022)

X2 = caravan_df.drop(["Purchase"], axis=1)
y2 = caravan_df.Purchase

X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.20)

clf = DecisionTreeClassifier(max_leaf_nodes=8, max_features=6, random_state=2022)

clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)
accuarcy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuarcy)


y_prob = clf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_prob)
print("AUC: ", roc_auc)

**3.** Now, train another **DecisionTreeClassifier** using **entropy** as the split criterion at each node to fit a classification tree with a **maximum depth** of $4$ and a **minimum sample leaf** of $3$ on the training data and find out the **testing accuracy** and **AUC**. (Use $2022$ as the random seed value)

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree

np.random.seed(2022)


criteria = "entropy"
max_depth = 4
min_sample_leaf = 3
clf_gini = DecisionTreeClassifier(
    criterion=criteria, max_depth=max_depth, min_samples_leaf=min_sample_leaf
)
clf_gini.fit(X_train, y_train)

y_pred = clf_gini.predict(X_test)
accuarcy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuarcy)


y_prob = clf_gini.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_prob)
print("AUC: ", roc_auc)

**4.** Compare the performance of the trained models in **Exercise 3** with **Exercise 2**.

*Compare your answer with the solution below*

```{toggle}
In terms of testing accuracy, the Exercise 2 model outperformed the Exercise 3 model, but accuracy is not the only metric to evaluate the models. When we look at the AUC score, which more comprehensively evaluates a model in imbalanced tasks, the Exercise 3 model outperforms the Exercise 2 model. Therefore, in this case we can see the Exercise 3 model is better than the Exercise 2 model.
```

**5.** Create a **confusion matrix** and a [**classification report**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) to help you evaluate the model you trained in **Exercise 3**.

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = clf_gini.predict(X_test)
cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred).T, index=["No", "Yes"], columns=["No", "Yes"]
)
print("Confusion matrix\n", cm)

print(classification_report(y_test, y_pred))

**a**. What is the testing accuracy of the model trained in **Exercise 3**? 

*Compare your answer with the solution below*

```{toggle}
The testing accuracy of the model trained in Exercise 3 is 0.94.
```

**b**. Does the model predicted well?

*Compare your answer with the solution below*

```{toggle}
If we only consider the test accuracy, we may conclude that the model learned the task effectively, but this is not the whole story. In fact, due to the class imbalance in the training data, this model is biased towards the "NO" class. If we look at the confusion matrix, we see that it predicts "NO" for almost all samples, and has a poor recall and precision rate for the "YES" class. Again, this shows that accuracy alone is also not always a good metric for evaluating models. Considering AUC, recall, and precision as well as showing the confusion matrix, we can get a much better picture.
```